In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

print("⚽ FEATURE ENGINEERING")
print("="*60)

# Charger les données
df = pd.read_csv('../data/raw/premier_league_matches.csv')
df['date'] = pd.to_datetime(df['date'])
print(f"✅ Loaded {len(df)} matches")
print(f"📅 From {df['date'].min().date()} to {df['date'].max().date()}")
print(f"⚽ {df['home_team'].nunique()} unique teams")
print(f"\n🎯 TARGET: Create features to predict match outcomes!")

⚽ FEATURE ENGINEERING
✅ Loaded 760 matches
📅 From 2023-08-11 to 2025-05-25
⚽ 23 unique teams

🎯 TARGET: Create features to predict match outcomes!


In [2]:
# ============================================
# FEATURE 1: RECENT FORM (Forme des 5 derniers matchs)
# ============================================

print("🔄 Calculating recent form (last 5 matches)...")

# On va calculer les points des 5 derniers matchs pour chaque équipe
# Victoire = 3 points, Nul = 1 point, Défaite = 0 points

def calculate_form(team, date, n_matches=5):
    """
    Calcule les points des N derniers matchs d'une équipe AVANT une date donnée
    """
    # Matchs de cette équipe (home ou away) AVANT cette date
    team_matches = df[
        ((df['home_team'] == team) | (df['away_team'] == team)) & 
        (df['date'] < date)
    ].sort_values('date', ascending=False).head(n_matches)
    
    if len(team_matches) == 0:
        return 0  # Début de saison, pas d'historique
    
    points = 0
    for _, match in team_matches.iterrows():
        if match['home_team'] == team:
            # Cette équipe jouait à domicile
            if match['home_score'] > match['away_score']:
                points += 3  # Victoire
            elif match['home_score'] == match['away_score']:
                points += 1  # Nul
            # Défaite = 0 points
        else:
            # Cette équipe jouait à l'extérieur
            if match['away_score'] > match['home_score']:
                points += 3  # Victoire
            elif match['away_score'] == match['home_score']:
                points += 1  # Nul
    
    return points

# Tester sur le premier match
test_match = df.iloc[50]  # Match random
print(f"\n🧪 TEST sur: {test_match['home_team']} vs {test_match['away_team']}")
print(f"Date: {test_match['date']}")

home_form = calculate_form(test_match['home_team'], test_match['date'])
away_form = calculate_form(test_match['away_team'], test_match['date'])

print(f"Home form (last 5): {home_form} points")
print(f"Away form (last 5): {away_form} points")
print(f"\n💡 Form difference: {home_form - away_form} (positive = home advantage)")

🔄 Calculating recent form (last 5 matches)...

🧪 TEST sur: Luton Town FC vs Wolverhampton Wanderers FC
Date: 2023-09-23 14:00:00+00:00
Home form (last 5): 0 points
Away form (last 5): 3 points

💡 Form difference: -3 (positive = home advantage)


In [4]:
# ============================================
# APPLIQUER LA FORME À TOUS LES MATCHS
# ============================================

print("⏳ Calculating form for all 760 matches...")
print("(This might take 30-60 seconds...)\n")

# Créer les colonnes
df['home_form'] = 0
df['away_form'] = 0

# Calculer pour chaque match
for idx, match in df.iterrows():
    df.loc[idx, 'home_form'] = calculate_form(match['home_team'], match['date'])
    df.loc[idx, 'away_form'] = calculate_form(match['away_team'], match['date'])
    
    # Progress bar simple
    if (idx + 1) % 100 == 0:
        print(f"✅ Processed {idx + 1}/760 matches...")

# Créer la feature "form difference"
df['form_diff'] = df['home_form'] - df['away_form']

print(f"\n🎉 DONE! Form calculated for all matches!")
print(f"\n📊 FORM STATISTICS:")
print(df[['home_form', 'away_form', 'form_diff']].describe())

print(f"\n👀 Sample with form:")
print(df[['home_team', 'away_team', 'home_form', 'away_form', 'form_diff']].head(10))

⏳ Calculating form for all 760 matches...
(This might take 30-60 seconds...)

✅ Processed 100/760 matches...
✅ Processed 200/760 matches...
✅ Processed 300/760 matches...
✅ Processed 400/760 matches...
✅ Processed 500/760 matches...
✅ Processed 600/760 matches...
✅ Processed 700/760 matches...

🎉 DONE! Form calculated for all matches!

📊 FORM STATISTICS:
        home_form   away_form   form_diff
count  760.000000  760.000000  760.000000
mean     6.540789    6.743421   -0.202632
std      3.721308    3.676828    5.003924
min      0.000000    0.000000  -12.000000
25%      4.000000    4.000000   -3.250000
50%      6.000000    7.000000    0.000000
75%      9.000000   10.000000    3.000000
max     15.000000   15.000000   15.000000

👀 Sample with form:
                   home_team                   away_team  home_form  \
0                 Burnley FC          Manchester City FC          0   
1                 Arsenal FC        Nottingham Forest FC          0   
2            AFC Bournemouth   

In [6]:
# Vérifier des matchs plus tard dans la saison
print("🔍 Checking form for matches later in the season:\n")

# Match 100 (environ journée 13)
print("Match 100:")
print(df.iloc[100][['date', 'home_team', 'away_team', 'home_form', 'away_form', 'form_diff']])

print("\n" + "="*50 + "\n")

# Match 300 (milieu de saison)
print("Match 300:")
print(df.iloc[300][['date', 'home_team', 'away_team', 'home_form', 'away_form', 'form_diff']])

print("\n" + "="*50 + "\n")

# Match 500 (fin de saison)
print("Match 500:")
print(df.iloc[500][['date', 'home_team', 'away_team', 'home_form', 'away_form', 'form_diff']])

print("\n" + "="*50 + "\n")

# Statistiques: combien de matchs ont form = 0?
zero_form = df[(df['home_form'] == 0) & (df['away_form'] == 0)]
print(f"📊 Matchs avec form=0 pour les deux équipes: {len(zero_form)}/760")
print(f"💡 C'est normal au début de saison!")

# Moyenne de la forme (excluant les 0)
non_zero = df[(df['home_form'] > 0) | (df['away_form'] > 0)]
print(f"\n📈 Stats pour matchs avec historique ({len(non_zero)} matchs):")
print(non_zero[['home_form', 'away_form', 'form_diff']].describe())

🔍 Checking form for matches later in the season:

Match 100:
date         2023-11-04 12:30:00+00:00
home_team                    Fulham FC
away_team         Manchester United FC
home_form                            5
away_form                            9
form_diff                           -4
Name: 100, dtype: object


Match 300:
date         2024-04-03 19:15:00+00:00
home_team           Manchester City FC
away_team               Aston Villa FC
home_form                           11
away_form                           10
form_diff                            1
Name: 300, dtype: object


Match 500:
date         2024-11-29 20:00:00+00:00
home_team    Brighton & Hove Albion FC
away_team               Southampton FC
home_form                           10
away_form                            3
form_diff                            7
Name: 500, dtype: object


📊 Matchs avec form=0 pour les deux équipes: 14/760
💡 C'est normal au début de saison!

📈 Stats pour matchs avec historique (746 matchs

In [7]:
# ============================================
# FEATURE 2: GOALS TRENDS (Derniers 5 matchs)
# ============================================

print("⚽ Calculating goals scored & conceded trends...\n")

def calculate_goals_stats(team, date, is_home, n_matches=5):
    """
    Calcule les buts marqués et encaissés des N derniers matchs
    
    Returns: (goals_scored, goals_conceded)
    """
    # Matchs AVANT cette date
    if is_home:
        team_matches = df[
            (df['home_team'] == team) & (df['date'] < date)
        ].sort_values('date', ascending=False).head(n_matches)
    else:
        team_matches = df[
            (df['away_team'] == team) & (df['date'] < date)
        ].sort_values('date', ascending=False).head(n_matches)
    
    if len(team_matches) == 0:
        return 0, 0
    
    goals_scored = 0
    goals_conceded = 0
    
    for _, match in team_matches.iterrows():
        if is_home:
            goals_scored += match['home_score']
            goals_conceded += match['away_score']
        else:
            goals_scored += match['away_score']
            goals_conceded += match['home_score']
    
    # Moyenne par match
    avg_scored = goals_scored / len(team_matches)
    avg_conceded = goals_conceded / len(team_matches)
    
    return avg_scored, avg_conceded

# Test rapide
test_match = df.iloc[300]  # Man City vs Aston Villa
print(f"🧪 TEST: {test_match['home_team']} vs {test_match['away_team']}")
print(f"Date: {test_match['date']}\n")

home_scored, home_conceded = calculate_goals_stats(
    test_match['home_team'], 
    test_match['date'], 
    is_home=True
)
away_scored, away_conceded = calculate_goals_stats(
    test_match['away_team'], 
    test_match['date'], 
    is_home=False
)

print(f"Man City (home) last 5 home matches:")
print(f"  ⚽ Scored: {home_scored:.2f} goals/match")
print(f"  🛡️  Conceded: {home_conceded:.2f} goals/match")

print(f"\nAston Villa (away) last 5 away matches:")
print(f"  ⚽ Scored: {away_scored:.2f} goals/match")
print(f"  🛡️  Conceded: {away_conceded:.2f} goals/match")

print(f"\n💡 Offensive advantage: {home_scored - away_scored:.2f}")
print(f"💡 Defensive advantage: {away_conceded - home_conceded:.2f}")

⚽ Calculating goals scored & conceded trends...

🧪 TEST: Manchester City FC vs Aston Villa FC
Date: 2024-04-03 19:15:00+00:00

Man City (home) last 5 home matches:
  ⚽ Scored: 1.40 goals/match
  🛡️  Conceded: 0.40 goals/match

Aston Villa (away) last 5 away matches:
  ⚽ Scored: 2.20 goals/match
  🛡️  Conceded: 0.80 goals/match

💡 Offensive advantage: -0.80
💡 Defensive advantage: 0.40


In [9]:
# ============================================
# APPLIQUER GOALS STATS À TOUS LES MATCHS
# ============================================

print("⏳ Calculating goals stats for all matches...")
print("(This will be FASTER - ~30 seconds)\n")

# Créer les colonnes
df['home_goals_scored_avg'] = 0.0
df['home_goals_conceded_avg'] = 0.0
df['away_goals_scored_avg'] = 0.0
df['away_goals_conceded_avg'] = 0.0

# Calculer pour chaque match
for idx, match in df.iterrows():
    # Home team
    h_scored, h_conceded = calculate_goals_stats(
        match['home_team'], 
        match['date'], 
        is_home=True
    )
    df.loc[idx, 'home_goals_scored_avg'] = h_scored
    df.loc[idx, 'home_goals_conceded_avg'] = h_conceded
    
    # Away team
    a_scored, a_conceded = calculate_goals_stats(
        match['away_team'], 
        match['date'], 
        is_home=False
    )
    df.loc[idx, 'away_goals_scored_avg'] = a_scored
    df.loc[idx, 'away_goals_conceded_avg'] = a_conceded
    
    if (idx + 1) % 100 == 0:
        print(f"✅ Processed {idx + 1}/760...")

# Créer features dérivées
df['offensive_strength_diff'] = df['home_goals_scored_avg'] - df['away_goals_scored_avg']
df['defensive_strength_diff'] = df['away_goals_conceded_avg'] - df['home_goals_conceded_avg']

print(f"\n🎉 DONE!\n")
print("📊 GOALS FEATURES STATISTICS:")
print(df[['home_goals_scored_avg', 'away_goals_scored_avg', 
          'offensive_strength_diff', 'defensive_strength_diff']].describe())

print(f"\n👀 Sample:")
print(df[['home_team', 'away_team', 
          'home_goals_scored_avg', 'away_goals_scored_avg',
          'offensive_strength_diff']].iloc[300:305])

⏳ Calculating goals stats for all matches...
(This will be FASTER - ~30 seconds)

✅ Processed 100/760...
✅ Processed 200/760...
✅ Processed 300/760...
✅ Processed 400/760...
✅ Processed 500/760...
✅ Processed 600/760...
✅ Processed 700/760...

🎉 DONE!

📊 GOALS FEATURES STATISTICS:
       home_goals_scored_avg  away_goals_scored_avg  offensive_strength_diff  \
count             760.000000             760.000000                760.00000   
mean                1.618575               1.413004                  0.20557   
std                 0.820154               0.693515                  0.98485   
min                 0.000000               0.000000                 -3.00000   
25%                 1.000000               1.000000                 -0.40000   
50%                 1.600000               1.400000                  0.20000   
75%                 2.200000               1.800000                  0.80000   
max                 5.000000               4.000000                  4.00000  

In [10]:
# ============================================
# TARGET VARIABLE: Match Result
# ============================================

print("🎯 Creating TARGET variable (match result)...\n")

def get_result(row):
    """
    H = Home Win
    D = Draw
    A = Away Win
    """
    if row['home_score'] > row['away_score']:
        return 'H'
    elif row['home_score'] < row['away_score']:
        return 'A'
    else:
        return 'D'

df['result'] = df.apply(get_result, axis=1)

print("✅ Target created!\n")
print("📊 DISTRIBUTION DES RÉSULTATS:")
result_counts = df['result'].value_counts()
print(result_counts)
print(f"\nPourcentages:")
print((result_counts / len(df) * 100).round(2))

print(f"\n👀 Sample avec toutes les features:")
print(df[['home_team', 'away_team', 
          'home_form', 'away_form', 'form_diff',
          'offensive_strength_diff', 'defensive_strength_diff',
          'home_score', 'away_score', 'result']].iloc[300:305])

🎯 Creating TARGET variable (match result)...

✅ Target created!

📊 DISTRIBUTION DES RÉSULTATS:
result
H    330
A    255
D    175
Name: count, dtype: int64

Pourcentages:
result
H    43.42
A    33.55
D    23.03
Name: count, dtype: float64

👀 Sample avec toutes les features:
              home_team             away_team  home_form  away_form  \
300  Manchester City FC        Aston Villa FC         11         10   
301        Liverpool FC   Sheffield United FC         13          2   
302          Chelsea FC  Manchester United FC          9          7   
303   Crystal Palace FC    Manchester City FC          5         11   
304      Aston Villa FC          Brentford FC          7          3   

     form_diff  offensive_strength_diff  defensive_strength_diff  home_score  \
300          1                     -0.8                      0.4           4   
301         11                      1.4                      0.8           3   
302          2                      0.0                    

In [11]:
# ============================================
# SAUVEGARDER LE DATASET AVEC FEATURES
# ============================================

print("💾 Saving dataset with features...\n")

# Sélectionner les colonnes importantes
columns_to_save = [
    'date', 'home_team', 'away_team',
    'home_score', 'away_score', 'matchday',
    'home_form', 'away_form', 'form_diff',
    'home_goals_scored_avg', 'home_goals_conceded_avg',
    'away_goals_scored_avg', 'away_goals_conceded_avg',
    'offensive_strength_diff', 'defensive_strength_diff',
    'result'
]

df_features = df[columns_to_save].copy()

# Sauvegarder
output_path = '../data/processed/matches_with_features.csv'
df_features.to_csv(output_path, index=False)

print(f"✅ Dataset saved to: {output_path}")
print(f"📦 Shape: {df_features.shape}")
print(f"📊 Features created: {df_features.shape[1] - 6} (excluding metadata)")

print(f"\n🎯 READY FOR MACHINE LEARNING!")
print(f"\n📈 Next steps:")
print("  1. Train/test split")
print("  2. Train models (Logistic Regression, Random Forest, XGBoost)")
print("  3. Evaluate & achieve 72%+ accuracy!")

print(f"\n🔥 Feature Engineering COMPLETE! 🔥")

💾 Saving dataset with features...

✅ Dataset saved to: ../data/processed/matches_with_features.csv
📦 Shape: (760, 16)
📊 Features created: 10 (excluding metadata)

🎯 READY FOR MACHINE LEARNING!

📈 Next steps:
  1. Train/test split
  2. Train models (Logistic Regression, Random Forest, XGBoost)
  3. Evaluate & achieve 72%+ accuracy!

🔥 Feature Engineering COMPLETE! 🔥
